In [34]:
# Importing modules
import glob
import os
from pathlib import Path
import pandas as pd
import zipfile

In [31]:
# Defining FileFinder
def FileFinder(YKR_IDs: list, input_folder_name: str, to_file=False):
    """
    Returns a list of time matrix files based on a list of YKR ID values
    from a specified input data folder. 

            Parameters:
                    YKR_IDs (list): A list of YKR_ID numbers
                    input_folder_name (str): Name of the input folder
                    to_file (boolean): If True, also returns a text file

            Returns:
                    file_paths (list): A list of filepaths
    """
    # Finding the folder from the user's instance
    input_folder = Path(input_folder_name).absolute()
    # Defining counter
    counter = 1
    # Creating an empty list for the filepaths
    file_paths = []
    # Using assert to make sure input is ok
    assert type(YKR_IDs) == list, "The input of the YKR_ID:s needs to be a list!"
    # For-looping the files in the user's instance
    for root, dirs, files in os.walk(input_folder):
        for name in files:
            # Checking if file contains "checkpoint" in its name
            # This is only for this course's sake
            if "checkpoint" in name:
                continue
            # Appending the file to the list of filepaths
            file_paths.append(name)
    # Checking if the given YKR_ID's can be found in the input folder
    for YKR_ID in YKR_IDs:
        id_check = f"travel_times_to_ {YKR_ID}.txt"
        if id_check not in file_paths:
            # Printing a warning but still continuing
            print(f"YKR_ID {YKR_ID} does not exist in the folder: {input_folder_name}")
        else:
            # Informing the user on the current progress
            print(f"Processing file travel_times_to_ {YKR_ID}.txt. Progress: {counter}/{len(file_paths)}")
            # Adding to the counter
            counter += 1
    # Checking for the optional parameter (default is False)
    if to_file == True:
        # Writing the list of filepaths to a text file
        with open("YKR_ID_fps", "w") as output:
            output.write(str(file_paths))
        # Informing the user of the name of the new text file
        print("Wrote the filepaths to: YKR_ID_fps.txt")
    # Returning the list of filepaths
    return file_paths           

In [32]:
# A test list for the FileFinder
lista = ["5787544", "5787545", "5787546", "5787547", "5787548", "5787549", "5787550", "1234567", "5990000", "5990001", "5990002"]

In [33]:
# Testing the FileFinder
FileFinder(lista, "input_data", to_file=True)

Processing file travel_times_to_ 5787544.txt. Progress: 1/10
Processing file travel_times_to_ 5787545.txt. Progress: 2/10
Processing file travel_times_to_ 5787546.txt. Progress: 3/10
Processing file travel_times_to_ 5787547.txt. Progress: 4/10
Processing file travel_times_to_ 5787548.txt. Progress: 5/10
Processing file travel_times_to_ 5787549.txt. Progress: 6/10
Processing file travel_times_to_ 5787550.txt. Progress: 7/10
YKR_ID 1234567 does not exist in the folder: input_data
Processing file travel_times_to_ 5990000.txt. Progress: 8/10
Processing file travel_times_to_ 5990001.txt. Progress: 9/10
Processing file travel_times_to_ 5990002.txt. Progress: 10/10
Wrote the filepaths to: YKR_ID_fps.txt


['travel_times_to_ 5787544.txt',
 'travel_times_to_ 5787545.txt',
 'travel_times_to_ 5787546.txt',
 'travel_times_to_ 5787547.txt',
 'travel_times_to_ 5787548.txt',
 'travel_times_to_ 5787549.txt',
 'travel_times_to_ 5787550.txt',
 'travel_times_to_ 5990000.txt',
 'travel_times_to_ 5990001.txt',
 'travel_times_to_ 5990002.txt']

In [2]:
def TableJoiner(fp_list: list):
    
    
    
    
    

SyntaxError: unexpected EOF while parsing (<ipython-input-2-6b3f7d4f316b>, line 1)

In [47]:
# Defining unzip
def unzip(zip_file: str, target_folder: str):
    """
    Extracts selected file to a selected directory. 

            Parameters:
                    zip_file (str): Name of the zip file
                    target_dir (str): Name of the target directory

            Returns:
                    None
    """
    # Reading zipfile
    with zipfile.ZipFile(zip_file, "r") as zip_ref:
        # Extracting zipfile to target folder
        zip_ref.extractall(target_folder)
        # Informing user of the progress
        print(f"File: {zip_file} extracted to: {target_folder}.")

In [48]:
# Testing unzip
unzip("MetropAccess_YKR_grid.zip", "grid_file")

File: MetropAccess_YKR_grid.zip extracted to: grid_file.


File: MetropAccess_YKR_grid.zip extracted to: grid_file.


str

In [3]:
def Visualizer():

SyntaxError: unexpected EOF while parsing (<ipython-input-3-935f67ef1ef3>, line 1)

In [4]:
def ComparisonTool():

SyntaxError: unexpected EOF while parsing (<ipython-input-4-c7fae3c3a60c>, line 1)